# Art Geneneration Using Deep Learning (Neural Style Transfer)

<p>Deep Learning has been the most important important branch of machine learning and particularly Computer Vision which has led to the rise of self-driving cars and facial recognition. One of the most  fascinating application of Computer Vision using Convolutional Neural Networks is the generation of art images.The algorithm uses neural representations to separate and recombine content and style images ,providing a neural algorithm for the creation of artistic images.To understand how convolutional neural networks are able to generate art images, we need to understand how layers in convolutional neural networks work.</p>

## What are deep convolutional neural networks learning ?
<p>Inorder to understand what are convolutional neural networks learning we need to pick a unit in each layer and find 9 image patches that maximize the unit's activation hidden unit.</p>

![Layer 1](./images/layer1.png)

<p>The above image shows hidden units in layer 1 and if we take a look at the first 9 image patches in the top left corner in layer 1 we can see that the image patches are activated by a diagonal line in the image. We can also see that the 9 image patches below those first 9 image patches in the top left corner are activated by an orange color and that's what activates the hidden units.</p>

![Layer2](images/layer2.png)

<p>We can see that layer 2 is now detecting compex shapes and patterns we can also see that 9 image patches below those in the top left corner are also activated by an orange color.</p>

![Layer 3](images/layer3.png)

<p>In layer 3 things start to get a little fascinating we can see that the 9 image patches in the top left corner hidden unit are activated by sort of a honey comb structure.We can also notice that they is a hidden unit that is activated by sort of circle or car wheel structures and they is also a hidden unit that is activated by people. As we go deeper into the convnet the layers of the convnet will start to see larger parts of the image.</p>

![layer 4](images/layer4.png)

<p>In layer 4 we can see that the first 9 image patches in the top left corner are activated by dogs .They are also other hideen units activated by water,birds and animal legs.</p>

<p>Now that we have gained an intuition about what shallow and deepr layers of a convolutional neural network are learning let's now use this knowledge to understanf how neural style transfer works. If you want to gain more knowledge about how convnets learn you may read this research paper in the link below.</p>


[Understanding and visualizing convolutional neural networks paper](https://arxiv.org/abs/1311.2901)


# Neural Style Transfer
<p>Our problem formulation in neural style transfer is how do we generate a new art image given a content image p which we wish to keep and a style image a which we also want to keep </p>

<p>Content Image</p>

![Content Image](images/contentimage.png)

<p>Style Image</p>  

![Style Image](images/styleimage.png)
<p>Inorder to generate a new art image x , which is blended from the content image and the style image a ,we need to make sure that  x  returns the content features such as eyes ,ears and head from the content image p and also preserving the style from style image a.Thus we initialize our image x to be generated randomly into a white color image and we then use gradient decent the cost function of the generated image which we denote as J(G). J(G) the cost function of the generated image is the total cost function of the content image J(C) and that of the style image J(S)</p>

<p>The purpose of both the content loss and the style loss is to make sure that the generated image returns both features from the content image and the style image.The full explanation of the content loss and the style loss is below.</p>

In [313]:
import numpy as np
import pandas as pd
from PIL import Image
from keras import backend as K
from keras.preprocessing.image import load_img,img_to_array
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input
from scipy.optimize import fmin_l_bfgs_b
import time
import os



In [314]:
#Path for content image
cImPath=r'C:\Users\isheunesu\neuralstyle\DSC_0061_3.JPG'



In [315]:
#Path for the style image
sImPath=r'C:\Users\isheunesu\neuralstyle\pic.jpg'

In [316]:
#Path for the generated image
genImOutputPath=r"C:\Users\isheunesu\neuralstyle\generated.jpg"

In [317]:
targetHeight = 512
targetWidth = 512
targetSize = (targetHeight, targetWidth)

cImageOrig = Image.open(cImPath)
cImageSizeOrig = cImageOrig.size
cImage = load_img(path=cImPath, target_size=targetSize)
cImArr = img_to_array(cImage)
cImArr = K.variable(preprocess_input(np.expand_dims(cImArr, axis=0)), dtype='float32')

sImage = load_img(path=sImPath, target_size=targetSize)
sImArr = img_to_array(sImage)
sImArr = K.variable(preprocess_input(np.expand_dims(sImArr, axis=0)), dtype='float32')

gIm0 = np.random.randint(256, size=(targetWidth, targetHeight, 3)).astype('float64')
gIm0 = preprocess_input(np.expand_dims(gIm0, axis=0))

gImPlaceholder = K.placeholder(shape=(1, targetWidth, targetHeight, 3))

In [318]:
def get_feature_reps(x, layer_names, model):
    featMatrices = []
    for ln in layer_names:
        selectedLayer = model.get_layer(ln)
        featRaw = selectedLayer.output
        featRawShape = K.shape(featRaw).eval(session=tf_session)
        N_l = featRawShape[-1]
        M_l = featRawShape[1]*featRawShape[2]
        featMatrix = K.reshape(featRaw, (M_l, N_l))
        featMatrix = K.transpose(featMatrix)
        featMatrices.append(featMatrix)
    return featMatrices


## Content Loss
<p>The purpose of the content loss is to make sure that our generated image x retains some "global" charecteristics of the content image p.For example in our case we want to make sure that the generated image x looks like the house image in p. This means that shapes such windows and roof top ought to be recognizable.To archive this the content loss is defined as the mean squared error between the feature represantation p and x , respectively at a given layer l.For content loss layer l is neither too deep or too shallow.</p>

![Content Loss](images/contenttloss.png)

<p>The derivative of this loss with respect to the activations in layer l equals</p>

![Derivative Of The Content Loss](images/contentderivative.png)
<p>Remeber the image x we initialized randomly we can change that unitl it generates the same response in a certain layer of the CNN as the original image p by computing the gradient descent of the loss function.</p>

<ul>
    <li>F and P are matrices with a number of rows equal to N and a number of columns equal to M</li>
    <li>N is the number of filters in layer l and M is the number of spatial elements in the feature map (height times width) for layer l</li>
    <li>F contains the feature representation of x for layer l</li>
    <li>P contains the feature representation of p for layer l</li>
</ul>

In [319]:
def get_content_loss(F, P):
    cLoss = 0.5*K.sum(K.square(F - P))
    return cLoss

In [320]:
def get_Gram_matrix(F):
    G = K.dot(F, K.transpose(F))
    return G

## Style Loss
<p>The style loss is designed to preserve stylistic charecteristics of the style image, a .Rather than using the difference between the feature representations , the authors use the difference betweenthe Gram matrics from selected layers , where the Gram matrix is defined as </p>

![Gram Matrix](images/grammatrix.png)

<p>The Gram matrix is a square matrix that contains the dot products between each vectorized filter in layer l. The Gram matrix can therefore be thought of as a non-normalized correlation matrix for filters in layer l.Where A is the gram matrix of the style image a and G is the gram matrix of the generated image x</p>





![Total loss ](images/styleloss.png)




<p>Ascending layers in most Convolutional Neural Networks such as VGG have increasingly larger receptive fields . As this receptive field grows , more large scale charecteristics of the input image are preserved .Because of this multiple layers should be selected for the style to incorperate global stylistic qualities .To create a blending between these layers , we assign a weight w and define the total loss as</p>

![](images/stylelosstotal.png)


In [321]:
def get_style_loss(ws,Gs,As):
    sloss=K.variable(0.)
    for w,G,A in zip(ws,Gs,As):
        M_1=K.int_shape(G)[1]
        N_1=K.int_shape(G)[0]
        G_gram=get_Gram_matrix(G)
        A_gram=get_Gram_matrix(A)
        sloss+=w*0.25*K.sum(K.square(G_gram-A_gram))//(N_1**2*M_1**2)
        return sloss

## Total Loss
<p>To generate images that mix content of a photograph with the styling of a painting we jointly minimize the distance of a white noise image that we initialised randomly from the content representation of the photograph in one layer of the network and the style representation of the painting in a number of layers of the CNN.Let p be the photograph and a be the artwork.The loss we minimize is</p>

![](images/totalloss.png)

In [322]:
def get_total_loss(gImPlaceholder, alpha=1.0, beta=10000.0):
    F = get_feature_reps(gImPlaceholder, layer_names=[cLayerName], model=gModel)[0]
    Gs = get_feature_reps(gImPlaceholder, layer_names=sLayerNames, model=gModel)
    contentLoss = get_content_loss(F, P)
    styleLoss = get_style_loss(ws, Gs, As)
    totalLoss = alpha*contentLoss + beta*styleLoss
    return totalLoss

In [323]:
def calculate_loss(gImArr):
    """
    Calculate total loss using K.function
    """
    if gImArr.shape != (1, targetWidth, targetWidth, 3):
        gImArr = gImArr.reshape((1, targetWidth, targetHeight, 3))
    loss_fcn = K.function([gModel.input], [get_total_loss(gModel.input)])
    return loss_fcn([gImArr])[0].astype('float64')

In [324]:
def get_grad(gImArr):
    """
    Calculate the gradient of the loss function with respect to the generated image
    """
    if gImArr.shape != (1, targetWidth, targetHeight, 3):
        gImArr = gImArr.reshape((1, targetWidth, targetHeight, 3))
    grad_fcn = K.function([gModel.input], K.gradients(get_total_loss(gModel.input), [gModel.input]))
    grad = grad_fcn([gImArr])[0].flatten().astype('float64')
    return grad

In [325]:
def postprocess_array(x):
    # Zero-center by mean pixel
    if x.shape != (targetWidth, targetHeight, 3):
        x = x.reshape((targetWidth, targetHeight, 3))
    x[..., 0] += 103.939
    x[..., 1] += 116.779
    x[..., 2] += 123.68
    # 'BGR'->'RGB'
    x = x[..., ::-1]
    x = np.clip(x, 0, 255)
    x = x.astype('uint8')
    return x

In [326]:
def reprocess_array(x):
    x = np.expand_dims(x.astype('float64'), axis=0)
    x = preprocess_input(x)
    return x

In [327]:
#Save generated image
def save_original_size(x,target_size=cImageSizeOrig):
    xIm=Image.fromarray(x)
    xIm=xIm.resize(target_size)
    xIm.save(genImOutputPath)
    return xIm
    
    
    

In [ ]:
tf_session = K.get_session()
print('Generating Art')
cModel = VGG16(include_top=False, weights='imagenet', input_tensor=cImArr)
sModel = VGG16(include_top=False, weights='imagenet', input_tensor=sImArr)
gModel = VGG16(include_top=False, weights='imagenet', input_tensor=gImPlaceholder)
cLayerName = 'block4_conv2'#Content Layer
sLayerNames = [
                'block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1'
                #'block5_conv1'
                ]#Style Layers

P = get_feature_reps(x=cImArr, layer_names=[cLayerName], model=cModel)[0]
As = get_feature_reps(x=sImArr, layer_names=sLayerNames, model=sModel)
ws = np.ones(len(sLayerNames))/float(len(sLayerNames))

iterations = 600
x_val = gIm0.flatten()
start = time.time()
xopt, f_val, info= fmin_l_bfgs_b(calculate_loss, x_val, fprime=get_grad,
                            maxiter=iterations, disp=True)

xOut = postprocess_array(xopt)
xIm = save_original_size(xOut)
print( 'Image saved')
end = time.time()

print ('Time taken: {}'.format(end-start))

Generating Art


# References
https://medium.com/mlreview/making-ai-art-with-style-transfer-using-keras-8bb5fa44b216


[A Neural Algorithm of Artistic Style Paper](https://arxiv.org/abs/1508.06576)

https://github.com/llSourcell/AI_Artist

[Contact Me On Twitter](https://twitter.com)